<a href="https://colab.research.google.com/github/A5HU705H/Currency-Prediction-Challenge/blob/main/ens%C3%A9mbl%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the libraries and unzipping the data
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv
import os
import json
import shutil
from google.colab import drive
drive.mount('/content/drive')
! unzip /content/drive/MyDrive/currency-prediction-challenge.zip

In [ ]:
# copying train, test and json to main directory
from os import path
src1= '/content/data/train.csv'
src2= '/content/data/test.csv'
src3= '/content/data/cat_to_name.json'
dst= path.join('/content')
shutil.copy(src1, dst)
shutil.copy(src2, dst)
shutil.copy(src3, dst)

'/content/cat_to_name.json'

In [ ]:
# forming a dataframe containing the name of each of the training images and their corresponding labels
X_train= pd.read_csv('train.csv')
X_test= pd.read_csv('test.csv')

labelling= open('cat_to_name.json')
labels= labelling.read()
labels_dict= json.loads(labels)

Currency= X_train['currency'].copy()
Country= X_train['country'].copy()
X_train['value']=X_train['value'].str.cat(Currency, sep= ",")
X_train['value']=X_train['value'].str.cat(Country, sep= ",")

Labels_dict = labels_dict
labels_dict = { labels_dict[k]:k for k in labels_dict}
X_train['label']= X_train['value'].map(labels_dict)
x_train = pd.DataFrame([X_train['name'], X_train['label']]).transpose()

In [ ]:
# preprocessing setup used to set the data to be trained by each of the models:
! mkdir 'Train'
! mkdir 'Test'

from os import path
for i in Labels_dict:
  Path1= path.join('/content/Train', i.zfill(4))
  Path2= path.join('/content/Test', i.zfill(4))
  os.mkdir(Path1)
  os.mkdir(Path2)

dictionary= dict(x_train.values)
for j in dictionary:
  src= path.join('/content/data/train', j)
  dst= path.join('/content/Train', (dictionary.get(j)).zfill(4))
  shutil.copy(src, dst)

from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split=0.2,
    rescale=1./255)
training_set=train_datagen.flow_from_directory('/content/Train',target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=64, subset='training')
validation_set=train_datagen.flow_from_directory('/content/Train',target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=64, subset='validation')

In [ ]:
# defined architecture of each of the model used in ensemble
def inception_compile():
  pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
  )
  for layer in pre_trained_model.layers:
    layer.trainable= False

  for layer in pre_trained_model.layers[:-1]:
    layer.trainable= True

  x= tf.keras.layers.Dense(32, activation='relu')(pre_trained_model.output)
  x= tf.keras.layers.Flatten()(x)
  x= tf.keras.layers.Dense(211, activation='softmax')(x)

  model= tf.keras.Model(pre_trained_model.input, x)
  return model
def mobilenet_compile():
  from keras.layers import Dense, Input, Dropout
  from keras.models import Model

  input_tensor = Input(shape=(224, 224, 3))
  base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=input_tensor,
    input_shape=(224, 224, 3),
    pooling='avg')

  for layer in base_model.layers:
    layer.trainable = True

  x = Dense(512, activation='relu')(base_model.output)
  x = Dropout(.8)(x)

  predictions = Dense(211, activation='softmax')(x)

  model = Model(inputs=base_model.input, outputs=predictions)
  return model
def vgg19_compile():
  pre_trained_model = tf.keras.applications.vgg19.VGG19(
      include_top=False,
      weights='imagenet',
      input_shape=(224, 224, 3),
      pooling='max'
  )
  for layer in pre_trained_model.layers:
    layer.trainable= False

  x= tf.keras.layers.Flatten()(pre_trained_model.output)
  x= tf.keras.layers.Dense(211, activation='softmax')(x)

  model= tf.keras.Model(pre_trained_model.input, x)
  return model
def densenet_compile():
  pre_trained_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='max'
  )
  for layer in pre_trained_model.layers:
    layer.trainable= False

  for layer in pre_trained_model.layers[:-1]:
   layer.trainable= True

  x= tf.keras.layers.Dense(32, activation='relu')(pre_trained_model.output)
  x= tf.keras.layers.Flatten()(x)
  x= tf.keras.layers.Dense(211, activation='softmax')(x)

  model= tf.keras.Model(pre_trained_model.input, x)
  return model
def resnet_compile():
  pre_trained_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
  )
  for layer in pre_trained_model.layers:
   layer.trainable= True

  x= tf.keras.layers.Flatten()(pre_trained_model.output)
  x= tf.keras.layers.BatchNormalization()(x)
  x= tf.keras.layers.Dropout(0.5)(x)
  x= tf.keras.layers.Dense(211, activation='softmax')(x)
  model= tf.keras.Model(pre_trained_model.input, x)
  return model
def efficient_compile():
  pre_trained_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
  )
  for layer in pre_trained_model.layers:
   layer.trainable= False

  for layer in pre_trained_model.layers[:-1]:
   layer.trainable= True

  x= tf.keras.layers.Dense(32, activation='relu')(pre_trained_model.output)
  x= tf.keras.layers.Dropout(0.2)(x)
  x= tf.keras.layers.Dense(211, activation='softmax')(x)

  model= tf.keras.Model(pre_trained_model.input, x)
  return model

In [ ]:
# compiling the models
model1= inception_compile()
model2= vgg19_compile()
model3= densenet_compile()
model4= mobilenet_compile()
model5= resnet_compile()
model6= efficient_compile()

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:
# weights loaded from previously pre trained models
model1.load_weights('/content/drive/MyDrive/inception_enhanced.hdf5')           # 76.31 val_acc
model2.load_weights('/content/drive/MyDrive/vgg19.h5')                          # 77.01 val_acc
model3.load_weights('/content/drive/MyDrive/densenet.hdf5')                     # 83.34 val_acc
model4.load_weights('/content/drive/MyDrive/model.weights.best (2).hdf5')       # 82.23 val_acc
model5.load_weights('/content/drive/MyDrive/resnet50.hdf5')                     # 83.11 val_acc
model6.load_weights('/content/drive/MyDrive/efficientnetb0.hdf5')               # 81.22 val_acc

In [ ]:
# code to generate train dataset (altering path allows us to generate test dataset)
label1, label2, label3, label4, label5, label6= []
for i in x_train['name']:
  Path= path.join('/content/data/train', i)
  img = tf.io.read_file(Path)
  tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
  tensor = tf.image.resize(tensor, [224, 224])
  input_tensor = tf.expand_dims(tensor, axis=0)

  result1=model1.predict(input_tensor)
  result2=model2.predict(input_tensor)
  result3=model3.predict(input_tensor)
  result4=model4.predict(input_tensor)
  result5=model5.predict(input_tensor)
  result6=model6.predict(input_tensor)

  result1= result1[0]
  result2= result2[0]
  result3= result3[0]
  result4= result4[0]
  result5= result5[0]
  result6= result6[0]

  label1.append(result1)
  label2.append(result2)
  label3.append(result3)
  label4.append(result4)
  label5.append(result5)
  label6.append(result6)

a, b, c, d, e, f=[]
for i in range(0,len(label2[0])):
  a1, b1, c1, d1, e1, f1=[]
  for j in range(0,len(label2)):
    a1.append(label1[j][i])
    b1.append(label2[j][i])
    c1.append(label3[j][i])
    d1.append(label4[j][i])
    e1.append(label5[j][i])
    f1.append(label6[j][i])
  a.append(a1)
  b.append(b1)
  c.append(c1)
  d.append(d1)
  e.append(e1)
  f.append(f1)

dataset=pd.DataFrame()
for i in range(0,len(b)-1):
  dataset['vgg {}'.format(i+1)]=b[i]
for i in range(0,len(b)-1):
  dataset['densenet {}'.format(i+1)]=c[i]
for i in range(0,len(b)-1):
  dataset['mobilenet {}'.format(i+1)]=d[i]
for i in range(0,len(b)-1):
  dataset['resnet {}'.format(i+1)]=e[i]
for i in range(0,len(b)-1):
  dataset['inception {}'.format(i+1)]=a[i]
for i in range(0,len(b)-1):
  dataset['efficient {}'.format(i+1)]=f[i]

In [ ]:
# loading dataset 1(datasets generated using predict on train set) . . .
dataset= pd.read_csv('/content/drive/MyDrive/vgg_dense_mobile_res(dummy_removed).csv')

In [ ]:
# loading dataset 2(is used for the final calculation) . . .
dataset= pd.read_csv('/content/drive/MyDrive/vgg_dense_mobile_res_incep_eff(dummy_removed).csv')

In [ ]:
# consists of 1260 parameters, (removed the confidence of label 211 since it is a dummy variable.)
print(dataset)

         vgg 1     vgg 2     vgg 3     vgg 4     vgg 5         vgg 6  \
0     0.860291  0.001406  0.001041  0.001573  0.002455  3.723942e-07   
1     0.938574  0.002977  0.002688  0.000002  0.000035  1.371984e-08   
2     0.976812  0.000487  0.000982  0.000182  0.000116  9.110158e-07   
3     0.976812  0.000487  0.000982  0.000182  0.000116  9.110158e-07   
4     0.755424  0.016454  0.000162  0.000021  0.000001  4.150541e-08   
...        ...       ...       ...       ...       ...           ...   
6408  0.000113  0.000036  0.001285  0.000029  0.000132  3.007252e-07   
6409  0.000241  0.002314  0.001705  0.000865  0.000143  1.145769e-04   
6410  0.000020  0.000042  0.000168  0.000032  0.000018  2.166555e-08   
6411  0.000006  0.000006  0.001723  0.000003  0.000005  1.156740e-06   
6412  0.000034  0.000006  0.000564  0.000005  0.000003  8.519160e-06   

         vgg 7     vgg 8     vgg 9    vgg 10  ...  efficient 201  \
0     0.000738  0.002486  0.001530  0.000358  ...   3.707376e-05   

In [ ]:
# saving dataset. . .
dataset.to_csv('/content/drive/MyDrive/vgg_dense_mobile_res_incep_eff(dummy_removed).csv', index=False)

In [ ]:
# labels used for models that don't like one hot encoding
y1=[]
for i in x_train['label']:
  y1.append(int(i))

In [ ]:
# labels used for models that like one hot encoding
y= tf.keras.utils.to_categorical(x_train['label'])
y= np.delete(y, 0, axis=1)
print(y.shape)

(6413, 211)


In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size = 0.2, random_state = 0)
X0, X1, y0, y1 = train_test_split(dataset, y1, test_size = 0.2, random_state = 0)

In [ ]:
# ensemble Deep Learning
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dropout(0.4))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=211, activation='softmax'))

In [ ]:
# compiling the deep learning model
ann.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
161/161 [==============================] - 1s 3ms/step - loss: 5.1142 - accuracy: 0.0536
Epoch 2/50
161/161 [==============================] - 0s 3ms/step - loss: 3.6655 - accuracy: 0.2485
Epoch 3/50
161/161 [==============================] - 0s 3ms/step - loss: 2.1024 - accuracy: 0.6195
Epoch 4/50
161/161 [==============================] - 0s 3ms/step - loss: 1.3466 - accuracy: 0.7359
Epoch 5/50
161/161 [==============================] - 0s 3ms/step - loss: 1.0671 - accuracy: 0.7692
Epoch 6/50
161/161 [==============================] - 0s 3ms/step - loss: 0.9291 - accuracy: 0.7864
Epoch 7/50
161/161 [==============================] - 0s 3ms/step - loss: 0.8277 - accuracy: 0.8012
Epoch 8/50
161/161 [==============================] - 0s 3ms/step - loss: 0.7696 - accuracy: 0.8150
Epoch 9/50
161/161 [==============================] - 0s 3ms/step - loss: 0.7457 - accuracy: 0.8156
Epoch 10/50
161/161 [==============================] - 0s 3ms/step - loss: 0.6525 - accuracy: 0.8380

In [ ]:
#ensemble fitting (logistic)
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(random_state = 0)
classifier1.fit(X0, y0)

LogisticRegression(random_state=0)

In [ ]:
# deep learning prediction comparision
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = ann.predict(X_test)
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)
accuracy_score(y_test.argmax(axis=1),y_pred.argmax(axis=1))

[[7 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 6 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]


0.9563522992985191

In [ ]:
# non-one hot encoded prediction comparision
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[7 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 6 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]


0.961028838659392

In [ ]:
# one hot encoded prediction comparison
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)
accuracy_score(y_test, y_pred)

[[7 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 6 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]


0.9454403741231489

In [ ]:
# test dataset loading. . .
test_dataset= pd.read_csv('/content/drive/MyDrive/TestSet.csv')

In [ ]:
print(test_dataset)

            vgg 1         vgg 2         vgg 3         vgg 4         vgg 5  \
0    7.865998e-01  4.898704e-04  4.098047e-05  5.240663e-05  4.951632e-04   
1    6.990524e-05  3.008193e-07  4.227759e-06  3.921734e-06  8.552957e-08   
2    3.708693e-04  1.951009e-06  1.706128e-05  7.420514e-07  5.177032e-06   
3    3.311248e-04  1.738439e-03  4.614207e-03  2.662936e-03  1.125625e-04   
4    3.899672e-05  4.803655e-07  9.093321e-06  5.319938e-08  7.230418e-08   
..            ...           ...           ...           ...           ...   
839  2.633990e-05  4.355523e-07  2.394918e-07  2.712719e-08  1.986265e-07   
840  1.662544e-05  1.066535e-05  2.597549e-05  5.472966e-07  6.666833e-08   
841  8.335537e-03  5.022981e-03  1.036655e-04  1.497426e-03  3.162025e-03   
842  7.993731e-03  1.311836e-03  1.733450e-03  5.048872e-03  8.051062e-04   
843  4.905693e-10  2.102849e-10  1.211971e-12  3.200299e-13  2.429136e-08   

            vgg 6         vgg 7         vgg 8         vgg 9        vgg 10  

In [ ]:
# test dataset saving. . .
test_dataset.to_csv('/content/drive/MyDrive/TestSet.csv', index=False)

In [ ]:
# creating predictions from Logistic Regression ensemble
y_pred_1 = classifier.predict(test_dataset)

In [ ]:
y_pred_1 = tf.keras.utils.to_categorical(y_pred_1)
y_pred_1 = np.delete(y_pred_1, 0, axis=1)

In [ ]:
# creating predictions from Neural Network ensemble
y_pred = ann.predict(test_dataset)

In [ ]:
# combining the ensembling models to take average
y_pred = y_pred + y_pred_1

In [ ]:
print(y_pred)

[[1.9999846e+00 6.7485006e-12 1.0672779e-18 ... 3.6691786e-24
  8.0536741e-22 4.3845274e-09]
 [1.9550727e-07 1.5009165e-12 4.8172949e-14 ... 4.1322506e-13
  1.1748915e-09 1.6521388e-05]
 [2.6507449e-16 1.6262801e-18 9.0375318e-27 ... 2.1093465e-13
  1.2117259e-16 1.0044809e-06]
 ...
 [4.6714023e-04 1.7597940e-03 2.0400879e-04 ... 2.3836068e-05
  1.2218922e-04 1.1331978e+00]
 [2.3893446e-14 1.4372372e-13 7.8913910e-26 ... 4.1971191e-14
  1.7385234e-09 7.8000721e-07]
 [2.3936728e-22 3.6401286e-21 1.1311547e-11 ... 3.8520830e-13
  1.1515856e-14 6.6050783e-09]]


In [ ]:
# using test data to creat format which is submittable
x_test= pd.read_csv('test.csv')
x_test.insert(1, 'label', (y_pred.argmax(axis=1)+1), True)
print(x_test)

        name  label
0      1.jpg      1
1      2.jpg    191
2      3.jpg     11
3      4.jpg    145
4      5.jpg     48
..       ...    ...
839  840.jpg     80
840  841.jpg     53
841  842.jpg    211
842  843.jpg     33
843  844.jpg     94

[844 rows x 2 columns]


In [ ]:
# saving submission file
x_test.to_csv('/content/drive/MyDrive/submission.csv', index=False)